In [71]:
import requests
import pandas as pd
from pycoingecko import CoinGeckoAPI
import time
import datetime
import psycopg2
import datetime


In [88]:
now = datetime.datetime.now()

2020-10-09 19:51:59.877153


In [2]:
cg = CoinGeckoAPI()

In [99]:
#Connect the dbname to the name of the database we create
conn = psycopg2.connect("host=localhost dbname='task2Playground' user='postgres' password='###'")
cur = conn.cursor()


In [5]:
#creates a lists for naming each table, yearnFinance is created without a dash to aovid naming error
coins = ['ethereum', 'yearnFinance', 'chainlink', 'ethlend', 'uniswap', 'havven', 'uma', 'nxm', 'rarible']
#creates a list to store the correct api names to get data from coinGecko
coin = ['ethereum', 'yearn-finance', 'chainlink', 'ethlend', 'uniswap', 'havven', 'uma', 'nxm', 'rarible']

In [6]:
#Create each table in the databse
for i in range(len(coins)):
    cur.execute(f"""
    CREATE TABLE IF NOT EXISTS {coins[i]} (
    coin_id SERIAL PRIMARY KEY,
    date TIMESTAMP NOT NULL,
    price decimal NOT NULL
)
             """)

In [7]:
conn.commit()

In [98]:
while True:
    #gets the current price of all the coins in the list
    token_prices = cg.get_price(ids=coin, vs_currencies='usd')
    for x in range(len(coins)):
        #parses through the data to get the price of the current coin we are working with, converted to a float
        coin_price = float(token_prices[coin[x]]['usd'])
        #gets the current date and time and formats it neatly
        timeDate = now.strftime("%Y-%m-%d %H:%M:%S")
        #inserts the data for the selected coin into the correct database
        #found the TIMESTAMP keyword online and it seemed to help store the data correctly
        cur.execute(f"INSERT INTO {coins[x]} (date, price) VALUES (TIMESTAMP'{timeDate}',{coin_price})")
        conn.commit()
    #the sleep time can be adjusted to the corrected intervals of how often the data is updated
    time.sleep(10)
   

KeyboardInterrupt: 